# CE 295 Final Project
# EV Charging Schedule Optimization

## Generate Synthetic EV Vehicle + Driver Behavior Data

In [1]:
import pandas as pd
import numpy as np

### Vehicle Specifications

In [2]:
# SET NUMBER OF VEHICLES

veh_num = 10

#### Assume all vehicles are the same make and model
#### Tesla Model 3 Long Range

“Tesla Model 3 Long Range Dual Motor,” EV Database. https://ev-database.org/car/1992/Tesla-Model-3-Long-Range-Dual-Motor

“Onboard Charger | Tesla Support,” Tesla, 2024. https://www.tesla.com/support/charging/onboard-charger (accessed Apr. 04, 2024).

In [11]:
# Assume all vehicles are the same make and model
# Tesla Model 3 Long Range

veh_make = "Tesla"
veh_model = "Model 3 Long Range"
batt_cap = 75 # kWh
batt_chg_rate = 11 # kWh
fuel_econ = round(150/1000/0.621371,2) # [kWh/mi]  = 150 Wh/km / 1000 Wh/kWh / 0.621371 mi/km 

veh_pop = pd.DataFrame(columns = ["VehID","Make","Model","Battery_Capacity_kWh", "Charging_Rating_kW", "Fuel_Econ_kWh/mi"])

for i in range(1,veh_num+1):
    veh_pop.loc[len(veh_pop)] = [i, veh_make, veh_model, batt_cap, batt_chg_rate, fuel_econ]

veh_pop

,VehID,Make,Model,Battery_Capacity_kWh,Charging_Rating_kW,Fuel_Econ_kWh/mi
0,1,Tesla,Model 3 Long Range,75,11,0.24
1,2,Tesla,Model 3 Long Range,75,11,0.24
2,3,Tesla,Model 3 Long Range,75,11,0.24
3,4,Tesla,Model 3 Long Range,75,11,0.24
4,5,Tesla,Model 3 Long Range,75,11,0.24
5,6,Tesla,Model 3 Long Range,75,11,0.24
6,7,Tesla,Model 3 Long Range,75,11,0.24
7,8,Tesla,Model 3 Long Range,75,11,0.24
8,9,Tesla,Model 3 Long Range,75,11,0.24
9,10,Tesla,Model 3 Long Range,75,11,0.24


### Set up a function to generate random data along a normal distribution

In [12]:
# Define a function that can be used to generate random (normally distributed) data

def gen_synth_dat(lower, upper, samples, seed):
    ''' Input the lower and upper bounds for the normal distribution
        and the desired number of random samples to be generated'''
    
    # Approximate standard deviation as (upper-lower)/6 (capture 99.7% of distribution)
    std_dev = (upper - lower) / 6
    
    # Calculate the mean of the normal distribution as lower + 3*std_dev (or upper - 3*std_dev)
    norm_mean = lower + 3 * std_dev
    
    # Generate random values from a normal distribution
    np.random.seed(seed)
    rand_vals = np.random.normal(norm_mean, std_dev, samples)
    
    return rand_vals

### Randomize the Initial SOC for each vehicle at Hr 0

In [13]:
# Generate synthetic data points for initial state of charge for each vehicle
init_SOC_lower = 0.1
init_SOC_upper = 1
seed = 123
init_SOC_random = gen_synth_dat(init_SOC_lower, init_SOC_upper, veh_num, seed)

# round the SOC values to 2 decimal places for simplicity
init_SOC_random = [round(val, 2) for val in init_SOC_random]

veh_pop["Init_SOC"] = init_SOC_random

### Randomize AM commute hour and PM commute hour (times when vehicle is driven)

In [14]:
# Morning commute starts between 5AM and 9AM . Use randomized sampling of normal dist. to assign start time
am_commute_lower = 5  
am_commute_upper = 9
seed = 321
am_commute_random = gen_synth_dat(am_commute_lower, am_commute_upper, veh_num, seed)

# round to an integer
am_commute_random = [round(val) for val in am_commute_random]

veh_pop["AM_Commute_Hr"] = am_commute_random

In [15]:
# Evening commute starts between 3PM/15:00 and 7PM/19:00. Use randomized sampling of normal dist. to assign start time
pm_commute_lower = 15 
pm_commute_upper = 19
seed = 789
pm_commute_random = gen_synth_dat(pm_commute_lower, pm_commute_upper, veh_num, seed)

# round to an integer
pm_commute_random = [round(val) for val in pm_commute_random]

veh_pop["PM_Commute_Hr"] = pm_commute_random

### Set Commute Length

In [16]:
# Assume all have same commute length

commute_dist = 20 # mi per one-way trip

veh_pop["Commute_mi_per_trip"] = [commute_dist] * veh_num

### Randomize each driver's access to charging (home only, work only, always)

simplifying assumption: driver is either at home, work, or currently driving
The home and work chargers are identical models

Equal likelihood of access to home, work, or both chargers

In [17]:
# Define options
charge_access_options = ["home", "work", "both"]

# Randomly assign an option (corresponding to index in list) to each driver
np.random.seed(567)
charge_access_rand = [np.random.randint(0,3) for veh in range(veh_num)]

# Translate random integer to description of option
charge_access_rand_str = [charge_access_options[rand_opt] for rand_opt in charge_access_rand]

# Add to df
veh_pop["charge_access"] = charge_access_rand_str

## Review Dataset

In [18]:
veh_pop

,VehID,Make,Model,Battery_Capacity_kWh,Charging_Rating_kW,Fuel_Econ_kWh/mi,Init_SOC,AM_Commute_Hr,PM_Commute_Hr,Commute_mi_per_trip,charge_access
0,1,Tesla,Model 3 Long Range,75,11,0.24,0.39,7,16,20,home
1,2,Tesla,Model 3 Long Range,75,11,0.24,0.70,8,17,20,both
2,3,Tesla,Model 3 Long Range,75,11,0.24,0.59,7,17,20,home
3,4,Tesla,Model 3 Long Range,75,11,0.24,0.32,6,18,20,work
4,5,Tesla,Model 3 Long Range,75,11,0.24,0.46,6,17,20,work
5,6,Tesla,Model 3 Long Range,75,11,0.24,0.80,7,16,20,home
6,7,Tesla,Model 3 Long Range,75,11,0.24,0.19,6,17,20,home
7,8,Tesla,Model 3 Long Range,75,11,0.24,0.49,6,17,20,both
8,9,Tesla,Model 3 Long Range,75,11,0.24,0.74,7,16,20,work
9,10,Tesla,Model 3 Long Range,75,11,0.24,0.42,8,18,20,both


### Export to CSV

In [19]:
veh_pop.to_csv("Vehicle_Input_Data.csv")